In [90]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During

In [2]:
# Hugging Face API로 KoBERT 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-7niv12md/kobert-tokenizer_ec7a7579a9d5448eb43df753e08e0934
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-7niv12md/kobert-tokenizer_ec7a7579a9d5448eb43df753e08e0934
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=8300e500e003badddedd227d8975364a3e34daabff6870db36dfdf0939795dcf
  Stored in directory: /tmp/pip-ephem-wheel-cache-2owa1cub/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [91]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
!pip3 install kobert-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
#from transformers import BertTokenizer

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
# GPU 사용 확인
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [92]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
# ●
#bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [93]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [94]:
import pandas as pd 

path = './'
train_data = pd.read_csv(path + 'review_dataset.csv',encoding='utf-8-sig')

train_dataset = []
for sen, label in zip(train_data['content'], train_data['메뉴명']):
    data_train = []
    data_train.append(sen)
    data_train.append(str(label))

    train_dataset.append(data_train)

In [95]:
# 훈련, 테스트 데이터셋 분리 
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(train_dataset, test_size=0.25, random_state=0)

In [96]:
class BERTDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len, pad=True, pair=False):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair
        self.dataset = dataset

    def __getitem__(self, idx):
        if self.pair:
            text = self.dataset[idx][0]
            text_pair = self.dataset[idx][1]
        else:
            text = self.dataset[idx][0]

        inputs = self.tokenizer.encode_plus(
            text,
            text_pair if self.pair else None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=self.pad,
            truncation=True
        )

        item = {
            'input_ids': torch.tensor(inputs['input_ids']),
            'attention_mask': torch.tensor(inputs['attention_mask']),
            'token_type_ids': torch.tensor(inputs['token_type_ids'])
        }
        # print(torch.tensor(inputs['input_ids']).shape)
        # print(torch.tensor(inputs['attention_mask']).shape)
        # print(torch.tensor(inputs['token_type_ids']).shape)

        item['labels'] = int(self.dataset[idx][1])
        # item['labels'] = self.dataset[idx][1]

        # 패딩된 시퀀스/길이와 타입에 대한 내용/어텐션 마스크 시퀀스 세 가지 배열을 얻을 수 있도록 함
        padded_seq = torch.tensor(inputs['input_ids']).numpy()
        len_and_type = list(torch.tensor(inputs['token_type_ids']).shape) #dtype은 출력 안됨
        attention_mask_seq = torch.tensor(inputs['attention_mask']).numpy()

        return np.array(padded_seq), np.array(len_and_type), np.array(attention_mask_seq), item['labels']


    def __len__(self):
        return len(self.dataset)


In [113]:
# 데이터셋 생성
train_dataset = BERTDataset(dataset_train, tokenizer, max_len, True, False)
test_dataset = BERTDataset(dataset_test, tokenizer, max_len, True, False)

# 데이터로더 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [131]:
txt = tokenizer.decode(train_dataset[0][0])
txt

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [132]:
tokenizer.tokenize(txt)

['[CLS]',
 '▁카',
 '라',
 '멜',
 '▁마',
 '끼',
 '아',
 '또',
 '는',
 '▁부',
 '드',
 '럽',
 '고',
 '▁크',
 '리',
 '미',
 '한',
 '▁',
 '텍',
 '스',
 '처',
 '로',
 '▁한',
 '▁잔',
 '▁마',
 '시',
 '면',
 '▁즉각',
 '적인',
 '▁만족',
 '감을',
 '▁선사',
 '합니다',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [115]:
train_dataset[0]

(array([   2, 4635, 6003, 6194, 1907, 5649, 6797, 5980, 5760, 2423, 5920,
        6044, 5439, 4681, 6122, 6255, 7828,  517, 7620, 6664, 7416, 6079,
        4955, 3941, 1907, 6705, 6198, 4280, 7206, 1947, 5344, 2751, 7843,
          54,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1]),
 array([64]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 4)

In [116]:
dataset_train[0]

['카라멜 마끼아또는 부드럽고 크리미한 텍스처로 한 잔 마시면 즉각적인 만족감을 선사합니다.', '4']

In [117]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [118]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# model = BERTClassifier(bertmodel,  dr_rate=0.5)

In [119]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


In [120]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [121]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


In [122]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_loader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        # token_ids = token_ids.long()
        segment_ids = segment_ids.long().to(device)
        # segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long().to(device)
        # print(label)
        # label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_loader)):
        token_ids = token_ids.long().to(device)
        # token_ids = token_ids.long()
        segment_ids = segment_ids.long().to(device)
        # segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long().to(device)
        # label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


<ipython-input-122-00a7fd27a515>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_loader)):


  0%|          | 0/18 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.0514285564422607 train acc 0.0
epoch 1 train acc 0.1593364197530864


<ipython-input-122-00a7fd27a515>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_loader)):


  0%|          | 0/6 [00:00<?, ?it/s]

...

epoch 29 test acc 0.9836309523809524


  0%|          | 0/18 [00:00<?, ?it/s]

epoch 30 batch id 1 loss 0.003638974390923977 train acc 1.0
epoch 30 train acc 0.9982638888888888


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 30 test acc 0.9836309523809524


In [123]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, tokenizer, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        # token_ids = token_ids.long()
        # segment_ids = segment_ids.long()

        valid_length= valid_length
        label = label.long().to(device)
        # label = label.long()

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            print(i)
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("아아")
            elif np.argmax(logits) == 1:
                test_eval.append("따아")
            elif np.argmax(logits) == 2:
                test_eval.append("콜드브루")
            elif np.argmax(logits) == 3:
                test_eval.append("아이스 카라멜 마끼아또")
            elif np.argmax(logits) == 4:
                test_eval.append("카라멜마끼아또")    
            else:
                test_eval.append("None")
        print(test_eval)

In [124]:
predict("시원한 음료 먹고싶어")
predict("단 거 먹고싶어")
predict("카라멜")
predict("단 맛")
predict("콜드 브루")
predict("따뜻한")
predict("여름")
predict("카라멜의 향긋한 향이 풍부한 음료입니다")

tensor([-1.5173, -1.8197,  0.4474, -1.8823,  5.5541], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['카라멜마끼아또']
tensor([-1.5488, -1.7824,  1.2912, -2.1445,  4.8481], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['카라멜마끼아또']
tensor([-1.1048, -1.5924, -1.4436, -1.7362,  6.6219], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['카라멜마끼아또']
tensor([-1.6328, -1.5943,  2.2697, -2.4449,  3.9845], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['카라멜마끼아또']
tensor([-1.3131, -1.2243,  5.7505, -1.5298, -1.3050], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['콜드브루']
tensor([-1.7227,  4.0368,  0.1739, -1.8117,  0.2858], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['따아']
tensor([-1.4427, -0.8811,  1.1181, -2.0528,  4.2381], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['카라멜마끼아또']
tensor([-1.2272, -1.5925, -1.3311, -1.7362,  6.6637], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['카라멜마끼아또']


In [125]:
predict("너무 더운데 시원한 음료 마시고 싶어")

tensor([-1.7437, -1.6270,  1.8275, -1.9883,  4.2018], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['카라멜마끼아또']


In [128]:
predict("아이스 음료")

tensor([ 4.8641, -1.6990, -2.0773, -1.0394, -0.7715], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['아아']


In [127]:
predict("카라멜 들어간 거")

tensor([-1.1396, -1.6005, -1.4040, -1.7529,  6.6340], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['카라멜마끼아또']
